In [18]:
import matplotlib
matplotlib.use("Agg")
from neuralnilm.source import RealApplianceSource, MultiSource, SameLocation
import matplotlib.pyplot as plt
import numpy as np
import logging

In [19]:
MAX_TARGET_POWER = 3000

TARGET_APPLIANCE = 'microwave'

BUILDING_I = 5
SEQ_LENGTH = 128
TRAIN_BUILDINGS = [BUILDING_I]
VALIDATION_BUILDINGS = [BUILDING_I]
N_SEQ_PER_BATCH = 8

USE_SHORT_WINDOWS = False
if USE_SHORT_WINDOWS:
    WINDOW_PER_BUILDING = {
        1: ("2013-04-12", "2013-04-18"),
        2: ("2013-05-22", "2013-06-01"),
        3: ("2013-02-27", "2013-03-10"),
        4: ("2013-03-09", "2013-04-01"),
        5: ("2014-06-29", "2014-07-10")
    }
else:
    WINDOW_PER_BUILDING = {
        1: ("2013-03-17", "2014-12-01"),
        2: ("2013-05-22", "2013-10-01"),
        3: ("2013-02-27", "2013-04-01"),
        4: ("2013-03-09", "2013-09-20"),
        5: ("2014-06-29", "2014-08-27")
    }

INPUT_STATS = {
    'mean': np.array([ 297.87216187], dtype=np.float32),
    'std': np.array([ 374.43884277], dtype=np.float32)
}

In [20]:
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [21]:
source = SameLocation(
    logger=logger,
    filename='/data/mine/vadeec/merged/ukdale.h5',
    target_appliance=TARGET_APPLIANCE,
    window_per_building=WINDOW_PER_BUILDING,
    seq_length=SEQ_LENGTH,
    train_buildings=TRAIN_BUILDINGS,
    validation_buildings=VALIDATION_BUILDINGS,
    n_seq_per_batch=N_SEQ_PER_BATCH,
    load_mains=False,
    on_power_threshold=200,
    min_on_duration=12,
    min_off_duration=30
)

INFO:test:Setting window for building 5 to (start=2014-06-29, end=2014-08-27)
INFO:test:Loaded 22 microwave activations from house 5.


In [22]:

seq_lengths = [(activation.index[-1] - activation.index[0]).total_seconds() / 6 for activation in source.activations[BUILDING_I]]
print("Max number of samples:", max(seq_lengths))
print("Mean number of samples:", np.mean(seq_lengths))
print("Min number of samples:", np.min(seq_lengths))

Max number of samples: 104.0
Mean number of samples: 18.9545454545
Min number of samples: 4.0


In [5]:
import nilmtk
from datetime import timedelta

In [6]:

def plot_activation(activation_i):
    activation = source.activations[BUILDING_I][activation_i]
    border = timedelta(minutes=10)
    timeframe = nilmtk.TimeFrame(activation.index[0]-border, activation.index[1]+border)
    powers = {}
    source.dataset.set_window()
    elec = source.dataset.buildings[BUILDING_I].elec
    for meter in elec.submeters().meters:
        power = meter.power_series_all_data(sections=[timeframe])
        if power is not None and power.max() > 10:
            powers[meter.dominant_appliance().label(pretty=True)] = power
    
    fig, axes = plt.subplots(nrows=3, sharex=True)
    axes[0].set_title('Activation')
    axes[0].plot(activation.index, activation.values)

    axes[1].set_title('All active appliances')
    for meter_instance, power in powers.iteritems():
        axes[1].plot(power.index, power.values, label=meter_instance)
    axes[1].legend(fancybox=True, framealpha=0.5, prop={'size': 6})

    axes[2].set_title('Mains {}'.format(timeframe.start.date()))
    mains = elec.mains().power_series_all_data(sections=[timeframe], sample_period=6)
    if mains is not None:
        axes[2].plot(mains.index, mains.values)
    
    fig.tight_layout()
    fname = 'building{}_activation{}.png'.format(BUILDING_I, activation_i)
    plt.savefig(fname, dpi=300)
    fig.close()

In [7]:
from sys import stdout
n = len(source.activations[BUILDING_I])
for i in range(n):
    print("\r", i, "/", n, end="")
    stdout.flush()
    plot_activation(i)

 21 / 22

/usr/local/lib/python2.7/dist-packages/matplotlib/pyplot.py:424: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [8]:
elec = source.dataset.buildings[BUILDING_I].elec

In [9]:
elec['microwave'].power_series_all_data().max()

2098.0

In [10]:
source.activations[BUILDING_I][5].max()

2098.0